# Heart Disease Risk Prediction with Logistic Regression (Homework)

This notebook implements logistic regression from scratch (no scikit-learn) for the Heart Disease dataset (Kaggle/UCI). It respects and reuses what you've already done: data loading, target binarization, EDA, and outlier handling.

Objectives:
- Implement base functions (sigmoid, cost, gradients, gradient descent)
- Train and evaluate the model (train/test metrics)
- Visualize decision boundaries for feature pairs
- Add L2 regularization and compare results

Note about the dataset: the Kaggle file "neurocipher/heartdisease" contains 270 samples (not 303 as in the original UCI release). The target was already binarized to 1=Presence, 0=Absence in your previous work. We'll use those columns and keep original names.

## Step 1: Load and Prepare the Dataset
- Source: [Kaggle - neurocipher/heartdisease](https://www.kaggle.com/datasets/neurocipher/heartdisease)
- Target binarization already done (`Heart Disease`: Presence→1, Absence→0)
- EDA: summary, missing values, outliers (already completed)
- Split: 70/30 stratified
- Normalization: z-score on train; apply same parameters to test
- Select ≥6 features: `Age`, `Cholesterol`, `BP`, `Max HR`, `ST depression`, `Number of vessels fluro`

In [ ]:
# Imports and configuration
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
np.random.seed(42)

# Try to reuse 'df' if it exists from your previous work
try:
    df  # check reference
    print("Using existing DataFrame ('df') from previous steps.")
except NameError:
    print("'df' not found in the environment, attempting to download from Kaggle.")
    try:
        import kagglehub
        path = kagglehub.dataset_download("neurocipher/heartdisease")
        df = pd.read_csv(f"{path}/Heart_Disease_Prediction.csv")
        print("Dataset downloaded from Kaggle and loaded.")
        # Binarize if not already
        if df['Heart Disease'].dtype == object:
            df['Heart Disease'] = df['Heart Disease'].map({'Presence': 1, 'Absence': 0})
    except Exception as e:
        raise RuntimeError("Could not load the dataset. Make sure you've executed the prior loading cell or have the CSV locally.") from e

# Show data summary
print("Shape:", df.shape)
print(df[['Heart Disease']].head())

target_col = 'Heart Disease'
feature_cols = ['Age', 'Cholesterol', 'BP', 'Max HR', 'ST depression', 'Number of vessels fluro']

# Basic checks
assert target_col in df.columns, "Column 'Heart Disease' not found in the DataFrame."
for c in feature_cols:
    assert c in df.columns, f"Column '{c}' not found in the DataFrame."

In [ ]:
# Brief additional EDA (your main EDA was already done)
print("\nInfo:")
print(df.info())

print("\nDescription:")
display(df[feature_cols + [target_col]].describe())

# Class distribution
class_counts = df[target_col].value_counts()
print("\nClass distribution:")
print(class_counts)
plt.figure(figsize=(4,3))
sns.barplot(x=class_counts.index.astype(str), y=class_counts.values)
plt.title("Class Distribution (Heart Disease)")
plt.xlabel("Class")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

# Note: Kaggle dataset used has 270 entries and ~44% presence (as per your prior EDA).

### Helper functions: stratified split and normalization
- Stratified 70/30 split by class
- Standardization (z-score) on train and apply same parameters to test

In [ ]:
def stratified_train_test_split(X, y, test_size=0.3, random_state=42):
    """Stratified split without scikit-learn.
    Returns X_train, X_test, y_train, y_test
    """
    np.random.seed(random_state)
    classes = np.unique(y)
    train_idx = []
    test_idx = []
    for cls in classes:
        idx = np.where(y == cls)[0]
        np.random.shuffle(idx)
        n_test = int(len(idx) * test_size)
        test_idx.extend(idx[:n_test])
        train_idx.extend(idx[n_test:])
    # optional final shuffle
    train_idx = np.array(train_idx)
    test_idx = np.array(test_idx)
    return X[train_idx], X[test_idx], y[train_idx], y[test_idx]

def standardize_train_test(X_train, X_test):
    """Standardize X_train and apply the same parameters to X_test. Returns X_train_std, X_test_std, (mean, std)."""
    mean = X_train.mean(axis=0)
    std = X_train.std(axis=0)
    # avoid division by 0
    std[std == 0] = 1.0
    X_train_std = (X_train - mean) / std
    X_test_std = (X_test - mean) / std
    return X_train_std, X_test_std, (mean, std)

In [ ]:
# Prepare data (numpy matrices)
X = df[feature_cols].values.astype(float)
y = df[target_col].values.astype(int)

X_train, X_test, y_train, y_test = stratified_train_test_split(X, y, test_size=0.3, random_state=42)
X_train_std, X_test_std, scalers = standardize_train_test(X_train, X_test)

print("Shapes:", X_train_std.shape, X_test_std.shape, y_train.shape, y_test.shape)

## Step 2: Implement Logistic Regression (from scratch)
- Functions: sigmoid, cost (binary cross-entropy), gradients, GD
- Train on the full training set with α≈0.01, ≥1000 iterations
- Plot cost vs. iterations
- Predict and compute metrics on train/test (accuracy, precision, recall, F1)

In [ ]:
# From-scratch implementation
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def initialize_params(n_features):
    w = np.zeros((n_features,))
    b = 0.0
    return w, b

def predict_proba(X, w, b):
    return sigmoid(X.dot(w) + b)

def compute_cost_and_grads(X, y, w, b, reg_lambda=0.0):
    m = X.shape[0]
    y_hat = predict_proba(X, w, b)
    # cost (cross-entropy): avoid log(0)
    epsilon = 1e-8
    cost = -(1/m) * (np.sum(y*np.log(y_hat+epsilon) + (1-y)*np.log(1-y_hat+epsilon)))
    # L2 (do not regularize bias)
    if reg_lambda > 0:
        cost += (reg_lambda/(2*m)) * np.sum(w**2)
    # gradients
    dw = (1/m) * X.T.dot(y_hat - y)
    db = (1/m) * np.sum(y_hat - y)
    if reg_lambda > 0:
        dw += (reg_lambda/m) * w
    return cost, dw, db

def gradient_descent(X, y, alpha=0.01, num_iters=2000, reg_lambda=0.0, verbose=False):
    w, b = initialize_params(X.shape[1])
    costs = []
    for i in range(num_iters):
        cost, dw, db = compute_cost_and_grads(X, y, w, b, reg_lambda)
        w -= alpha * dw
        b -= alpha * db
        costs.append(cost)
        if verbose and (i % 200 == 0 or i == num_iters-1):
            print(f"Iter {i}: cost={cost:.4f}")
    return w, b, costs

def predict_labels(X, w, b, threshold=0.5):
    probs = predict_proba(X, w, b)
    return (probs >= threshold).astype(int)

def accuracy(y_true, y_pred):
    return (y_true == y_pred).mean()

def precision_recall_f1(y_true, y_pred):
    # positive class = 1
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1 = 2 * precision * recall / (precision + recall + 1e-8)
    return precision, recall, f1

In [ ]:
# Train on the full training set without regularization (lambda=0)
alpha = 0.01
num_iters = 2000
reg_lambda = 0.0

w, b, costs = gradient_descent(X_train_std, y_train, alpha=alpha, num_iters=num_iters, reg_lambda=reg_lambda, verbose=True)

# Plot cost vs iterations
plt.figure(figsize=(6,3))
plt.plot(costs)
plt.title("Cost vs Iterations (no regularization)")
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.tight_layout()
plt.show()

# Metrics on train and test
y_train_pred = predict_labels(X_train_std, w, b)
y_test_pred  = predict_labels(X_test_std,  w, b)

acc_train = accuracy(y_train, y_train_pred)
acc_test  = accuracy(y_test,  y_test_pred)
prec_train, rec_train, f1_train = precision_recall_f1(y_train, y_train_pred)
prec_test,  rec_test,  f1_test  = precision_recall_f1(y_test,  y_test_pred)

metrics_df = pd.DataFrame({
    'split': ['train', 'test'],
    'accuracy': [acc_train, acc_test],
    'precision': [prec_train, prec_test],
    'recall': [rec_train, rec_test],
    'f1': [f1_train, f1_test]
})
display(metrics_df)

print("\nCoefficients (w):")
for name, coef in zip(feature_cols, w):
    print(f"{name}: {coef:.4f}")
print(f"Bias (b): {b:.4f}")

### Comments on convergence and interpretation
- The cost curve decreases with iterations, indicating learning.
- The `w` coefficients show the effect (in logits) of each standardized feature on the presence probability.
- Metrics: note differences between train/test and potential overfitting if test drops significantly relative to train.

## Step 3: Visualize Decision Boundaries (feature pairs)
We train 2D models (no regularization) and plot the boundary: `w0*x0 + w1*x1 + b = 0` → a line in the plane of those two standardized features.

Pairs (≥3):
- Age vs Cholesterol
- BP vs Max HR
- ST depression vs Number of vessels fluro

In [ ]:
pairs = [
    ('Age', 'Cholesterol'),
    ('BP', 'Max HR'),
    ('ST depression', 'Number of vessels fluro')
]

def train_2d_and_plot(df, pair, target_col='Heart Disease', alpha=0.05, num_iters=3000):
    f1, f2 = pair
    # Extract matrices
    X = df[[f1, f2]].values.astype(float)
    y = df[target_col].values.astype(int)
    # Stratified split
    X_train, X_test, y_train, y_test = stratified_train_test_split(X, y, test_size=0.3, random_state=42)
    # Standardize
    X_train_std, X_test_std, scalers = standardize_train_test(X_train, X_test)
    # Train
    w, b, costs = gradient_descent(X_train_std, y_train, alpha=alpha, num_iters=num_iters, reg_lambda=0.0, verbose=False)
    # Metrics
    y_pred_test = predict_labels(X_test_std, w, b)
    acc = accuracy(y_test, y_pred_test)
    prec, rec, f1 = precision_recall_f1(y_test, y_pred_test)

    # Plot: scatter + boundary
    plt.figure(figsize=(5,4))
    # points
    plt.scatter(X_test_std[y_test==0][:,0], X_test_std[y_test==0][:,1], c='tab:blue', s=25, label='Absence (0)', alpha=0.7)
    plt.scatter(X_test_std[y_test==1][:,0], X_test_std[y_test==1][:,1], c='tab:red', s=25, label='Presence (1)', alpha=0.7)
    # boundary: w1*x + w2*y + b = 0 → y = -(w1/w2)x - b/w2
    x_line = np.linspace(X_test_std[:,0].min()-0.5, X_test_std[:,0].max()+0.5, 100)
    if abs(w[1]) > 1e-8:
        y_line = -(w[0]/w[1])*x_line - b/w[1]
        plt.plot(x_line, y_line, 'k--', label='Decision boundary')
    plt.title(f"Decision Boundary: {f1} vs {f2}\nacc={acc:.2f}, prec={prec:.2f}, rec={rec:.2f}, f1={f1:.2f}")
    plt.xlabel(f1 + " (std)")
    plt.ylabel(f2 + " (std)")
    plt.legend()
    plt.tight_layout()
    plt.show()
    return {
        'pair': pair,
        'w': w,
        'b': b,
        'acc': acc,
        'prec': prec,
        'rec': rec,
        'f1': f1,
        'cost_last': costs[-1]
    }

pair_results = []
for p in pairs:
    res = train_2d_and_plot(df, p, target_col=target_col, alpha=0.05, num_iters=3000)
    pair_results.append(res)

pd.DataFrame(pair_results)[['pair','acc','prec','rec','f1','cost_last']]

### Comments by pair (separability and nonlinearity)
- Age vs Cholesterol: often shows some linear separation when cholesterol is high.
- BP vs Max HR: interesting combination; max HR tends to be informative.
- ST depression vs Number of vessels fluro: frequently useful; vessel count and ST depression associate with presence.

Observe the boundary shape: if the class mix does not appear linear, 2D logistic regression may not fully capture it (nonlinear features or more dimensions might be needed).

## Step 4: L2 Regularization
- Add L2 term to the cost and gradients: `cost += λ/(2m)||w||²`; `dw += (λ/m)w`
- Try several λ: [0, 0.001, 0.01, 0.1, 1]
- Retrain the full model and evaluate metrics and ||w||
- Compare boundaries and costs for at least one pair (no reg vs reg)

In [ ]:
lambdas = [0.0, 0.001, 0.01, 0.1, 1.0]
alpha = 0.01
num_iters = 2000

lambda_rows = []
for lam in lambdas:
    w_lam, b_lam, costs_lam = gradient_descent(X_train_std, y_train, alpha=alpha, num_iters=num_iters, reg_lambda=lam, verbose=False)
    y_train_pred = predict_labels(X_train_std, w_lam, b_lam)
    y_test_pred  = predict_labels(X_test_std,  w_lam, b_lam)
    acc_train = accuracy(y_train, y_train_pred)
    acc_test  = accuracy(y_test,  y_test_pred)
    prec_train, rec_train, f1_train = precision_recall_f1(y_train, y_train_pred)
    prec_test,  rec_test,  f1_test  = precision_recall_f1(y_test,  y_test_pred)
    w_norm = np.linalg.norm(w_lam)
    lambda_rows.append({
        'lambda': lam,
        'train_acc': acc_train,
        'test_acc': acc_test,
        'train_f1': f1_train,
        'test_f1': f1_test,
        'w_norm': w_norm,
        'final_cost': costs_lam[-1]
    })

lambda_df = pd.DataFrame(lambda_rows)
display(lambda_df)

In [ ]:
# Visual comparison of costs and boundaries for one pair: Age vs Cholesterol (no reg vs reg)
pair = ('Age', 'Cholesterol')
f1, f2 = pair

# Pair data
X_pair = df[[f1, f2]].values.astype(float)
y_pair = df[target_col].values.astype(int)
X_pair_train, X_pair_test, y_pair_train, y_pair_test = stratified_train_test_split(X_pair, y_pair, test_size=0.3, random_state=42)
X_pair_train_std, X_pair_test_std, pair_scalers = standardize_train_test(X_pair_train, X_pair_test)

# Train without reg and with reg (λ=best from the table, or default 0.1)
best_lambda = lambda_df.sort_values('test_f1', ascending=False)['lambda'].iloc[0]
w0, b0, c0 = gradient_descent(X_pair_train_std, y_pair_train, alpha=0.05, num_iters=3000, reg_lambda=0.0)
wR, bR, cR = gradient_descent(X_pair_train_std, y_pair_train, alpha=0.05, num_iters=3000, reg_lambda=best_lambda)

plt.figure(figsize=(10,3))
plt.subplot(1,2,1)
plt.plot(c0, label='no reg (λ=0)')
plt.plot(cR, label=f'with reg (λ={best_lambda})')
plt.title("Costs (Age vs Cholesterol)")
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.legend()

plt.subplot(1,2,2)
plt.scatter(X_pair_test_std[y_pair_test==0][:,0], X_pair_test_std[y_pair_test==0][:,1], c='tab:blue', s=25, alpha=0.7, label='0')
plt.scatter(X_pair_test_std[y_pair_test==1][:,0], X_pair_test_std[y_pair_test==1][:,1], c='tab:red', s=25, alpha=0.7, label='1')
x_line = np.linspace(X_pair_test_std[:,0].min()-0.5, X_pair_test_std[:,0].max()+0.5, 100)
if abs(w0[1]) > 1e-8:
    y_line0 = -(w0[0]/w0[1])*x_line - b0/w0[1]
    plt.plot(x_line, y_line0, 'k--', label='λ=0')
if abs(wR[1]) > 1e-8:
    y_lineR = -(wR[0]/wR[1])*x_line - bR/wR[1]
    plt.plot(x_line, y_lineR, 'g--', label=f'λ={best_lambda}')
plt.title("Decision Boundary (Age vs Cholesterol)")
plt.xlabel("Age (std)")
plt.ylabel("Cholesterol (std)")
plt.legend()
plt.tight_layout()
plt.show()

# Compared metrics
y0_pred = predict_labels(X_pair_test_std, w0, b0)
yR_pred = predict_labels(X_pair_test_std, wR, bR)
m0 = precision_recall_f1(y_pair_test, y0_pred)
mR = precision_recall_f1(y_pair_test, yR_pred)
print(f"No reg (λ=0): acc={accuracy(y_pair_test, y0_pred):.3f}, prec={m0[0]:.3f}, rec={m0[1]:.3f}, f1={m0[2]:.3f}")
print(f"With reg (λ={best_lambda}): acc={accuracy(y_pair_test, yR_pred):.3f}, prec={mR[0]:.3f}, rec={mR[1]:.3f}, f1={mR[2]:.3f}")

print("\n||w|| no reg:", np.linalg.norm(w0))
print("||w|| with reg:", np.linalg.norm(wR))

### Regularization report
- λ vs metrics table (train/test) and norm of w
- With suitable L2 (e.g., λ≈0.01–0.1), generalization often improves (F1/accuracy on test), ||w|| decreases, and the cost stabilizes.

Example conclusion: "Optimal λ ≈ `best_lambda` improves test F1 vs. λ=0 by ~X% and reduces the norm of w, indicating less overfitting."

## Summary (Markdown)
- Download: Kaggle dataset "neurocipher/heartdisease" (270 samples), binary target (0/1). Distribution ~44% presence (per EDA).
- Preprocessing: no missing; outliers already explored; selected 6 features; stratified 70/30 split; z-score normalization.
- Training: logistic regression from scratch; cost decreases; train/test metrics reported.
- Boundaries: 3 pairs plotted with decision lines; separability discussed.
- L2 regularization: λ in [0, 0.001, 0.01, 0.1, 1]; observed trade-off and potential test improvement.

Next steps (optional):
- Pipeline and simulated deployment with Amazon SageMaker (not implemented here but you could package training in a container and a dummy endpoint).
- Nonlinear feature engineering or variable interactions to improve boundaries.